In [13]:
# Import required packages
import numpy as np
import cv2
import zipfile
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression

import tensorflow as tf
import tensorflow.keras as K
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from keras.preprocessing import image
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
import numpy as np

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input, Dropout, Flatten, concatenate
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D

import time
import pickle

from tensorflow.keras import datasets, layers, models


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# [DO NOT MODIFY THIS CELL]

# load the images
n_img = 50000
n_noisy = 40000
n_clean_noisy = n_img - n_noisy
imgs = np.empty((n_img,32,32,3))

# for i in range(n_img):
#     img_fn = f'../data/train_data/images/{i+1:05d}.png' # I changed the path
#     imgs[i,:,:,:]=cv2.cvtColor(cv2.imread(img_fn),cv2.COLOR_BGR2RGB)


# import zipfile
a = {}
zip_file = zipfile.ZipFile('/content/drive/MyDrive/data/train_data/imgs.npy.zip')
imgs = np.load(zip_file.open(zip_file.namelist()[0]))

# load the labels
clean_labels = np.genfromtxt('/content/drive/MyDrive/data/train_data/clean_labels.csv', delimiter=',', dtype="int8") # I changed the path
noisy_labels = np.genfromtxt('/content/drive/MyDrive/data/train_data/noisy_labels.csv', delimiter=',', dtype="int8") # I changed the path

# imgs

In [6]:
cleaned_labels = np.genfromtxt('/content/drive/MyDrive/Colab Notebooks/5243/cleaned_labels.csv', delimiter=',', dtype="int8") # I changed the path
cleaned_labels

array([6, 9, 9, ..., 9, 1, 1], dtype=int8)

In [10]:
cnn2 = models.Sequential([
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])
cnn2.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [11]:
# reduce learning rate when val_accuracy has stopped improving
lr_reduce = K.callbacks.ReduceLROnPlateau(monitor='val_accuracy',
                                          factor=0.6,
                                          patience=2,
                                          verbose=1,
                                          mode='max',
                                          min_lr=1e-7)
# stop training when val_accuracy has stopped improving
early_stop = K.callbacks.EarlyStopping(monitor='val_accuracy',
                                       patience=3,
                                       verbose=1,
                                       mode='max')
# callback to save the Keras model and (best) weights obtained on an epoch basis. here, the trained (compiled) model is saved in the current working directory as ''
checkpoint = K.callbacks.ModelCheckpoint('label_correction_model',
                                         monitor='val_accuracy',
                                         verbose=1,
                                         save_weights_only=False,
                                         save_best_only=True,
                                         mode='max',
                                         save_freq='epoch')

In [12]:
X = imgs/255.0

In [17]:
# document the start time
start = time.time()

# Fit the model using all data
history = cnn2.fit(X, 
                  cleaned_labels,
                  epochs = 10, 
                  validation_split = 0.2,
                  shuffle=True,
                  callbacks=[lr_reduce, early_stop, checkpoint],
                  verbose=1)

# document the training stop time
stop = time.time()
cnn2_trainingtime = stop - start
print(f"Training time: {cnn2_trainingtime}s") # prints: Training time

# Save the entire model as a SavedModel.
cnn2.save(f'/content/drive/MyDrive/Colab Notebooks/5243/saved_model/model_II')

#import pickle
with open('/content/drive/MyDrive/Colab Notebooks/5243/model_II_trainingtime', 'wb') as fp:
    pickle.dump(cnn2_trainingtime, fp)
with open('/content/drive/MyDrive/Colab Notebooks/5243/model_II_historyfit', 'wb') as fp:
    pickle.dump(history, fp)

Epoch 1/10
1250/1250 [==============================] - ETA: 0s - loss: 0.5207 - accuracy: 0.8189
Epoch 1: val_accuracy did not improve from 0.70070
1250/1250 [==============================] - 56s 45ms/step - loss: 0.5207 - accuracy: 0.8189 - val_loss: 0.9459 - val_accuracy: 0.6925 - lr: 6.0000e-04
Epoch 2/10
1249/1250 [============================>.] - ETA: 0s - loss: 0.4920 - accuracy: 0.8287
Epoch 2: val_accuracy did not improve from 0.70070
1250/1250 [==============================] - 56s 45ms/step - loss: 0.4920 - accuracy: 0.8287 - val_loss: 0.9755 - val_accuracy: 0.6932 - lr: 6.0000e-04
Epoch 3/10
1249/1250 [============================>.] - ETA: 0s - loss: 0.4648 - accuracy: 0.8383
Epoch 3: val_accuracy improved from 0.70070 to 0.70130, saving model to label_correction_model
INFO:tensorflow:Assets written to: label_correction_model/assets
1250/1250 [==============================] - 55s 44ms/step - loss: 0.4648 - accuracy: 0.8382 - val_loss: 0.9510 - val_accuracy: 0.7013 - lr:

In [18]:
# remember to delete this
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

(X_train, y_train), (X_test,y_test) = datasets.cifar10.load_data()

y_train = y_train.reshape(-1,) 
y_test = y_test.reshape(-1,)
def plot_sample(X, y, index):
    plt.figure(figsize = (15,2))
    plt.imshow(X[index])
    plt.xlabel(classes[y[index]])
X_train = X_train / 255.0
X_test = X_test / 255.0
cnn2.evaluate(X_test,y_test)

313/313 [==============================] - 4s 14ms/step - loss: 1.1262 - accuracy: 0.6869


[1.1261730194091797, 0.6869000196456909]